## Libraries and Configuration

In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm.notebook import tqdm

from pytz import timezone
import datetime as dt

from f1_score import f1_loss

import random

In [3]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
seed_everything(25)

In [4]:
from models.model_sy import resnetbase as MaskModel
from datasets.dataset_sy import MaskBaseDataset as MaskDataset
from trans.trans_sy import basic_train_trans as TrainTrans
from trans.trans_sy import basic_test_trans as TestTrans

CLASS_NUM = 18
NUM_WORKERS = 4
BATCH_SIZE = 32
NUM_EPOCH = 10
SAVE_INTERVAL = 3

load_path = ''

comment = ''

## Load Data & Model

In [5]:
c = ''
log = []

test_dir = '/opt/ml/input/data/train'
eval_dir = '/opt/ml/input/data/eval'
save_dir = '/opt/ml/image-classification-level1-25/save/'
now = (dt.datetime.now().astimezone(timezone("Asia/Seoul")).strftime("%Y%m%d_%H%M%S"))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = MaskModel(CLASS_NUM)
if load_path : model.load_state_dict(torch.load(load_path))    
model = model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optm = torch.optim.Adam(model.parameters())

TrainTransform = TrainTrans()
TestTransfrom = TestTrans()

dataset = MaskDataset(
    img_dir=os.path.join(test_dir, 'images')
)

# train dataset과 validation dataset을 8:2 비율로 나눕니다.
n_val = int(len(dataset) * 0.2)
n_train = len(dataset) - n_val
dataset_train_mask, dataset_test_mask = data.random_split(dataset, [n_train, n_val])

# 각 dataset에 augmentation 함수를 설정합니다.
dataset_train_mask.dataset.set_transform(TrainTransform)
dataset_test_mask.dataset.set_transform(TestTransfrom)

dataloader_train_mask = DataLoader(dataset=dataset_train_mask,
                                      batch_size=BATCH_SIZE,
                                      num_workers=NUM_WORKERS,
                                      shuffle = True
                                      )
dataloader_test_mask = DataLoader(dataset=dataset_test_mask,
                                      batch_size=BATCH_SIZE,
                                      num_workers=NUM_WORKERS,
                                      shuffle = False
                                      )

dataloaders = {
        "train": dataloader_train_mask,
        "test": dataloader_test_mask
    }

## Log

In [6]:
log.append(f'{c:#^80}')
log.append(f'  [Comment]')
log.append(f'{comment}')
log.append(f'{c:#^80}')
log.append(c); log.append(c); log.append(c)

log.append(f'Model         : {model.__class__.__name__}')
log.append(f'  load_state  : {load_path}')
log.append(f'Dataset       : {dataset_train_mask.__class__.__name__}')
log.append(f'  train_len    {len(dataset_train_mask):>10}')
log.append(f'  test_len     {len(dataset_test_mask):>10}')
log.append(f'Train_trans   : {TrainTrans.__name__}')
log.append(f'Test_trans    : {TestTrans.__name__}')
log.append(f'Start_Date    : {now}')
log.append(f'Device        : {device}')
log.append(f'CLASS_NUM     : {CLASS_NUM}')
log.append(f'NUM_WORKERS   : {NUM_WORKERS}')
log.append(f'BATCH_SIZE    : {BATCH_SIZE}')
log.append(f'NUM_EPOCH     : {NUM_EPOCH}')
log.append(f'SAVE_INTERVAL : {SAVE_INTERVAL}')


for line in log:
    print(line)
    
log.append(c); log.append(c); log.append(c)

################################################################################
  [Comment]

################################################################################



Model         : resnetbase
  load_state  : 
Dataset       : Subset
  train_len         14428
  test_len           3607
Train_trans   : basic_train_trans
Test_trans    : basic_test_trans
Start_Date    : 20210827_162709
Device        : cuda:0
CLASS_NUM     : 18
NUM_WORKERS   : 4
BATCH_SIZE    : 32
NUM_EPOCH     : 10
SAVE_INTERVAL : 3


## Train

In [8]:
from sklearn.metrics import f1_score
# SAMPLE 복붙
best_test_accuracy = 0.
best_test_loss = float('inf')
best_f1 = 0.

for epoch in range(NUM_EPOCH):
    for phase in ["train", "test"]:
        running_loss = 0.
        running_acc = 0.
        running_f1 = 0.
        
        if phase == "train":
            model.train()
        elif phase == "test":
            model.eval() 
            
        for idx, (images, labels) in enumerate(pbar := tqdm(dataloaders[phase]), start = 1):
            images, labels = images.to(device), labels.to(device)

            optm.zero_grad()
            
            with torch.set_grad_enabled(phase == "train"):
                logits = model(images)
                _, preds = torch.max(logits, 1)
                loss = loss_fn(logits, labels)
                if phase == "train":
                    loss.backward()  # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
                    optm.step()  # 계산된 gradient를 가지고 모델 업데이트
            
            running_loss += loss.item() * images.size(0)
            running_acc += torch.sum(preds == labels.data)
            #running_f1 += f1_loss(labels.data, preds)
            running_f1 += f1_score(labels.data.cpu().numpy(),preds.cpu().numpy(),average='macro')
            pbar.set_description(f"loss : {running_loss/(idx*BATCH_SIZE):.3f}, acc : {running_acc/(idx*BATCH_SIZE):.3f}, f1 : {running_f1/(idx*BATCH_SIZE):.3f}")
    
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)
        epoch_f1 = running_f1 / len(dataloaders[phase])
        
        log.append(f"[{phase.upper():<5}] Epoch {epoch:0>3d} // (avg) Loss : {epoch_loss:.3f}, Accuracy : {epoch_acc:.3f}, F1 : {epoch_f1:.3f}")
        print(log[-1])
        
        if phase == "test":
            if best_test_accuracy < epoch_acc:
                best_test_accuracy = epoch_acc
            if best_test_loss > epoch_loss:
                best_test_loss = epoch_loss
            if best_f1 < epoch_f1:
                best_f1 = epoch_f1
            if epoch % SAVE_INTERVAL == 0:
                torch.save(model.state_dict(), os.path.join(save_dir, f'{now}_{model.__class__.__name__}_epoch_{epoch:0>3d}.pt'))

torch.save(model.state_dict(), os.path.join(save_dir, f'{now}_{model.__class__.__name__}_finish_{NUM_EPOCH:0>3d}.pt'))


log.append(c)
print(log[-1])   
log.append(c)
print(log[-1])  
log.append(c)
print(log[-1])  
log.append(f'{c:#^80}')
print(log[-1])            
log.append(f':::학습종료:::')
print(log[-1])
log.append(f"최고 accuracy : {best_test_accuracy}, 최저 loss : {best_test_loss}, 최고 F1 : {best_f1}")
print(log[-1])
log.append(f'{c:#^80}')
print(log[-1]) 


[TRAIN] Epoch 000 // (avg) Loss : 0.376, Accuracy : 0.870, F1 : 0.765



[TEST ] Epoch 000 // (avg) Loss : 0.359, Accuracy : 0.879, F1 : 0.764



[TRAIN] Epoch 001 // (avg) Loss : 0.241, Accuracy : 0.918, F1 : 0.838



[TEST ] Epoch 001 // (avg) Loss : 0.286, Accuracy : 0.899, F1 : 0.814



[TRAIN] Epoch 002 // (avg) Loss : 0.171, Accuracy : 0.939, F1 : 0.881


...


[TEST ] Epoch 007 // (avg) Loss : 0.117, Accuracy : 0.967, F1 : 0.936



[TRAIN] Epoch 008 // (avg) Loss : 0.070, Accuracy : 0.975, F1 : 0.954



[TEST ] Epoch 008 // (avg) Loss : 0.166, Accuracy : 0.944, F1 : 0.897



[TRAIN] Epoch 009 // (avg) Loss : 0.021, Accuracy : 0.994, F1 : 0.989



[TEST ] Epoch 009 // (avg) Loss : 0.055, Accuracy : 0.983, F1 : 0.965



################################################################################
:::학습종료:::
최고 accuracy : 0.9830884337425232, 최저 loss : 0.05510356387762249, 최고 F1 : 0.964548610127511
################################################################################


## Test

In [11]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(eval_dir, 'info.csv'))
image_dir = os.path.join(eval_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    transforms.Resize((512, 384), Image.BILINEAR),
    # transforms.CenterCrop(300),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in tqdm(loader):
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

In [12]:
# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(save_dir, f'{now}_result.csv'), index=False)
log.append(f'test inference is done!')
print(log[-1])
log.append(c)
print(log[-1])
log.append(f'{c:-^80}')
print(log[-1])
log.append(c)
print(log[-1])



# log 저장
with open(os.path.join(save_dir, f'{now}.log'), "w") as f:
    now = (dt.datetime.now().astimezone(timezone("Asia/Seoul")).strftime("%Y%m%d_%H%M%S"))
    log.append(f'Finish_Date    : {now}')
    print(log[-1])
    for line in log: 
        f.write(line+'\n')

test inference is done!

--------------------------------------------------------------------------------

Finish_Date    : 20210827_112135
